In [1]:
import torchvision
from torchvision.datasets import CIFAR10

In [2]:
import torch
import torch.nn as nn

lr=0.1 # learning rate 0.1
test=False # train 인지 test 인지
batch_size=128 # training 데이터셋 배치 사이즈
batch_size_test=100 # test 데이터셋 배치 사이즈
num_workers=4

device = 'cuda' if torch.cuda.is_available() else 'cpu' # GPU 사용

In [3]:
import torchvision.transforms as transforms

transforms_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset_train=CIFAR10(root='./data/', train=True, download=True, transform=transforms_train)

Files already downloaded and verified


In [4]:
transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset_test = CIFAR10(root='./data', train=False, download=True, transform=transforms_test)

Files already downloaded and verified


In [5]:
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size_test, shuffle=False, num_workers=num_workers)

In [8]:
dataset_train.data.shape

(50000, 32, 32, 3)

In [6]:
classes = dataset_train.classes
print(classes)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [8]:
from model import resnet
model = resnet()
model = model.to(device)
print(model)

Resnet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace=True)
  (layer_2n): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), 

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr, momentum=0.9, weight_decay=1e-4)

def train(epoch, global_steps):
    model.train()

    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        global_steps += 1
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    
    acc = 100 * correct / total
    print(f'train epoch: {epoch} [{batch_idx+1} / {len(train_loader)}] ')
    print(f'loss: {train_loss / (batch_idx + 1)} | acc: {acc}')

    return global_steps


In [10]:
def test(epoch, best_acc):
    model.eval()

    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    acc = 100 * correct / total
    print(f'test epoch: {epoch} [{batch_idx+1} / {len(test_loader)}] ')
    print(f'loss: {test_loss / (batch_idx + 1)} | acc: {acc}')

    if (acc > best_acc):
        best_acc = acc
    
    return best_acc

In [10]:
best_acc = 0
epoch = 0
global_steps = 0

while True:
    epoch += 1
    global_steps = train(epoch, global_steps)
    best_acc = test(epoch, best_acc)
    print(f'best test acc: {best_acc}')

    if global_steps > 64000:
        break

train epoch: 1 [391 / 391] 
loss: 1.890470641653251 | acc: 30.484
test epoch: 1 [100 / 100] 
loss: 1.562194539308548 | acc: 42.29
best test acc: 42.29
train epoch: 2 [391 / 391] 
loss: 1.3909131876952814 | acc: 49.098
test epoch: 2 [100 / 100] 
loss: 1.237586503624916 | acc: 55.34
best test acc: 55.34
train epoch: 3 [391 / 391] 
loss: 1.05346372289121 | acc: 62.558
test epoch: 3 [100 / 100] 
loss: 0.9235042214393616 | acc: 67.65
best test acc: 67.65
train epoch: 4 [391 / 391] 
loss: 0.8406071752843345 | acc: 70.48
test epoch: 4 [100 / 100] 
loss: 1.0228724491596222 | acc: 67.03
best test acc: 67.65
train epoch: 5 [391 / 391] 
loss: 0.7110887036451599 | acc: 75.284
test epoch: 5 [100 / 100] 
loss: 0.8357703596353531 | acc: 72.81
best test acc: 72.81
train epoch: 6 [391 / 391] 
loss: 0.6311729858293558 | acc: 77.982
test epoch: 6 [100 / 100] 
loss: 0.6497384083271026 | acc: 77.92
best test acc: 77.92
train epoch: 7 [391 / 391] 
loss: 0.5722666374405326 | acc: 80.216
test epoch: 7 [100 / 

In [13]:
train_loader.dataset.data.shape

(50000, 32, 32, 3)

In [11]:
best_acc = 0
epoch = 0
global_steps = 0

while True:
    epoch += 1
    global_steps = train(epoch, global_steps)
    best_acc = test(epoch, best_acc)
    print(f'best test acc: {best_acc}')

    if global_steps > 64000:
        break

RuntimeError: running_mean should contain 16 elements not 32